In [2]:
pip install filterpy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110541 sha256=8ef2c24b2d78e9d0c5dacb50eb81955b7c7aca48e1653cc49fc7b4dd669d9203
  Stored in directory: c:\users\nakul\appdata\local\pip\cache\wheels\79\33\43\53b597b8f63de80842202a5fed633eea6f5ce3e3f6c6efbab8
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'filterpy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'filterpy'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
import time
from datetime import datetime
from scipy.spatial import distance
from filterpy.kalman import KalmanFilter
import os
from IPython.display import clear_output, Image, display
import matplotlib.pyplot as plt

print("All libraries imported successfully!")


In [23]:
# ================================
# CONFIGURATION
# ================================

CONFIG = {
    # Video Input
    'VIDEO_PATH': 'airport footage.mp4',  # Change to your video path
    'OUTPUT_PATH': 'output_video.mp4',
    'DISPLAY_WIDTH': 1280,  # Display width in notebook
    
    # Detection Settings
    'MODEL': 'yolov8n.pt',  # Will auto-download on first run
    'CONFIDENCE_THRESHOLD': 0.5,
    'PERSON_CLASS': 0,
    'LUGGAGE_CLASSES': [24, 28, 26],  # backpack, suitcase, handbag
    
    # Tracking Settings
    'MAX_AGE': 30,  # Frames to keep lost tracks
    'MIN_HITS': 3,   # Frames to confirm track
    'IOU_THRESHOLD': 0.3,
    
    # Crowd Density
    'GRID_SIZE': (5, 5),  # Divide frame into grid
    'DENSITY_CRITICAL': 1.0,  # persons/m²
    'PIXELS_PER_METER': 50,   # Calibration (adjust for your camera)
    
    # Abandoned Luggage Detection
    'OWNER_DISTANCE_THRESHOLD': 150,  # pixels (≈5 meters)
    'STATIONARY_TIME': 30,  # seconds (30 frames at 1 FPS check)
    'MIN_LUGGAGE_SIZE': 20,  # pixels
    
    # Optical Flow
    'ENABLE_OPTICAL_FLOW': True,
    'FLOW_THRESHOLD_LOW': 0.5,  # Bottleneck detection
    'FLOW_THRESHOLD_HIGH': 3.0,  # Panic detection
    
    # Display
    'SHOW_BBOXES': True,
    'SHOW_TRAJECTORIES': True,
    'SHOW_HEATMAP': True,
    'SHOW_FLOW': True,
    'PROCESS_EVERY_N_FRAMES': 1,  # Process every N frames (1=all frames)
}

# Create output directory
os.makedirs('output', exist_ok=True)
os.makedirs('output/alerts', exist_ok=True)

print("Configuration loaded!")
print(f"Video: {CONFIG['VIDEO_PATH']}")
print(f"Model: {CONFIG['MODEL']}")


Configuration loaded!
Video: airport footage.mp4
Model: yolov8n.pt


In [24]:
# ================================
# HELPER FUNCTIONS
# ================================

def compute_iou(box1, box2):
    """Compute Intersection over Union"""
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2
    
    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

def get_center(bbox):
    """Get center point of bounding box"""
    x1, y1, x2, y2 = bbox
    return (int((x1 + x2) / 2), int((y1 + y2) / 2))

def euclidean_distance(point1, point2):
    """Calculate Euclidean distance between two points"""
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def draw_text_with_background(img, text, position, font_scale=0.6, thickness=2, 
                               text_color=(255, 255, 255), bg_color=(0, 0, 0)):
    """Draw text with background for better visibility"""
    font = cv2.FONT_HERSHEY_SIMPLEX
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    
    x, y = position
    cv2.rectangle(img, (x, y - text_height - baseline), 
                  (x + text_width, y + baseline), bg_color, -1)
    cv2.putText(img, text, (x, y), font, font_scale, text_color, thickness)

def apply_colormap_to_heatmap(heatmap):
    """Apply color mapping to heatmap"""
    normalized = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
    colored = cv2.applyColorMap(normalized.astype(np.uint8), cv2.COLORMAP_JET)
    return colored

print("Helper functions defined!")


Helper functions defined!


In [25]:
# ================================
# SIMPLE TRACKER (ByteTrack-style)
# ================================

class SimpleTracker:
    """Simple multi-object tracker"""
    
    def __init__(self, max_age=30, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.tracks = {}
        self.next_id = 1
        self.frame_count = 0
        
    def update(self, detections):
        """
        Update tracker with new detections
        
        Args:
            detections: List of [x1, y1, x2, y2, confidence, class_id]
        
        Returns:
            List of active tracks: [(track_id, bbox, class_id), ...]
        """
        self.frame_count += 1
        
        # If no existing tracks, create new ones
        if not self.tracks:
            for det in detections:
                self.tracks[self.next_id] = {
                    'bbox': det[:4],
                    'class': int(det[5]),
                    'hits': 1,
                    'age': 0,
                    'last_seen': self.frame_count
                }
                self.next_id += 1
            return [(tid, track['bbox'], track['class']) 
                    for tid, track in self.tracks.items()]
        
        # Match detections to existing tracks using IoU
        matched = set()
        new_detections = []
        
        for det in detections:
            best_iou = 0
            best_track_id = None
            
            for track_id, track in self.tracks.items():
                if track_id in matched:
                    continue
                iou = compute_iou(det[:4], track['bbox'])
                if iou > self.iou_threshold and iou > best_iou:
                    best_iou = iou
                    best_track_id = track_id
            
            if best_track_id is not None:
                # Update existing track
                self.tracks[best_track_id]['bbox'] = det[:4]
                self.tracks[best_track_id]['hits'] += 1
                self.tracks[best_track_id]['age'] = 0
                self.tracks[best_track_id]['last_seen'] = self.frame_count
                matched.add(best_track_id)
            else:
                # New detection
                new_detections.append(det)
        
        # Create new tracks for unmatched detections
        for det in new_detections:
            self.tracks[self.next_id] = {
                'bbox': det[:4],
                'class': int(det[5]),
                'hits': 1,
                'age': 0,
                'last_seen': self.frame_count
            }
            self.next_id += 1
        
        # Age unmatched tracks
        tracks_to_remove = []
        for track_id, track in self.tracks.items():
            if track_id not in matched:
                track['age'] += 1
                if track['age'] > self.max_age:
                    tracks_to_remove.append(track_id)
        
        # Remove old tracks
        for track_id in tracks_to_remove:
            del self.tracks[track_id]
        
        # Return active tracks (with minimum hits)
        active_tracks = []
        for track_id, track in self.tracks.items():
            if track['hits'] >= self.min_hits:
                active_tracks.append((track_id, track['bbox'], track['class']))
        
        return active_tracks

print("✅ Tracker class defined!")


✅ Tracker class defined!


In [26]:
# ================================
# CROWD DENSITY ANALYZER
# ================================

class CrowdDensityAnalyzer:
    """Analyze crowd density and movement"""
    
    def __init__(self, frame_shape, grid_size=(5, 5), pixels_per_meter=50):
        self.frame_height, self.frame_width = frame_shape[:2]
        self.grid_rows, self.grid_cols = grid_size
        self.pixels_per_meter = pixels_per_meter
        
        # Calculate grid cell dimensions
        self.cell_height = self.frame_height // self.grid_rows
        self.cell_width = self.frame_width // self.grid_cols
        
        # Occupancy grid for heatmap
        self.occupancy_grid = np.zeros((self.grid_rows, self.grid_cols), dtype=np.float32)
        
        # Previous frame for optical flow
        self.prev_gray = None
        
        print(f"📊 Grid initialized: {self.grid_rows}x{self.grid_cols}")
        print(f"📏 Cell size: {self.cell_width}x{self.cell_height} pixels")
    
    def update_density(self, person_centers):
        """Update density grid with person positions"""
        # Accumulate positions in grid cells
        for cx, cy in person_centers:
            grid_x = min(int(cx / self.cell_width), self.grid_cols - 1)
            grid_y = min(int(cy / self.cell_height), self.grid_rows - 1)
            self.occupancy_grid[grid_y, grid_x] += 1
    
    def get_density_heatmap(self, frame_shape):
        """Generate density heatmap"""
        # Resize grid to frame size
        heatmap = cv2.resize(self.occupancy_grid, 
                            (frame_shape[1], frame_shape[0]), 
                            interpolation=cv2.INTER_LINEAR)
        
        # Apply Gaussian blur for smooth heatmap
        heatmap = cv2.GaussianBlur(heatmap, (51, 51), 0)
        
        # Normalize and apply colormap
        if heatmap.max() > 0:
            heatmap_colored = apply_colormap_to_heatmap(heatmap)
        else:
            heatmap_colored = np.zeros(frame_shape, dtype=np.uint8)
        
        return heatmap_colored
    
    def calculate_optical_flow(self, gray_frame):
        """Calculate dense optical flow"""
        if self.prev_gray is None:
            self.prev_gray = gray_frame
            return None
        
        # Farneback optical flow
        flow = cv2.calcOpticalFlowFarneback(
            self.prev_gray, gray_frame,
            None, 
            pyr_scale=0.5,
            levels=3,
            winsize=15,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0
        )
        
        self.prev_gray = gray_frame
        return flow
    
    def visualize_flow(self, flow, frame_shape):
        """Visualize optical flow as HSV"""
        if flow is None:
            return np.zeros(frame_shape, dtype=np.uint8)
        
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        hsv = np.zeros(frame_shape, dtype=np.uint8)
        hsv[..., 0] = angle * 180 / np.pi / 2  # Hue = direction
        hsv[..., 1] = 255  # Saturation
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        
        flow_bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        return flow_bgr
    
    def check_overcrowding(self):
        """Check for overcrowded zones"""
        alerts = []
        
        for i in range(self.grid_rows):
            for j in range(self.grid_cols):
                count = self.occupancy_grid[i, j]
                
                # Calculate area in square meters
                cell_area_pixels = self.cell_width * self.cell_height
                cell_area_m2 = cell_area_pixels / (self.pixels_per_meter ** 2)
                
                # Calculate density (persons per m²)
                density = count / cell_area_m2 if cell_area_m2 > 0 else 0
                
                if density > CONFIG['DENSITY_CRITICAL']:
                    alerts.append({
                        'zone': (i, j),
                        'density': density,
                        'count': int(count),
                        'level': 'CRITICAL'
                    })
        
        return alerts
    
    def reset_grid(self):
        """Reset occupancy grid"""
        self.occupancy_grid.fill(0)

print("✅ Crowd density analyzer defined!")


✅ Crowd density analyzer defined!


In [27]:
# ================================
# ABANDONED LUGGAGE DETECTOR
# ================================

class AbandonedLuggageDetector:
    """Detect abandoned luggage"""
    
    def __init__(self, distance_threshold=150, time_threshold=30, fps=30):
        self.distance_threshold = distance_threshold
        self.time_threshold = time_threshold
        self.fps = fps
        
        # Track luggage and their owners
        self.luggage_tracks = {}  # {luggage_id: {'owner_id': None, 'frames_alone': 0, ...}}
        self.owner_luggage_pairs = {}  # {owner_id: [luggage_ids]}
        
        self.abandoned_alerts = []
        
    def update(self, person_tracks, luggage_tracks, frame_count):
        """
        Update abandoned luggage detection
        
        Args:
            person_tracks: List of (track_id, bbox, class)
            luggage_tracks: List of (track_id, bbox, class)
            frame_count: Current frame number
        """
        # Extract centers
        person_centers = {tid: get_center(bbox) for tid, bbox, _ in person_tracks}
        luggage_centers = {tid: get_center(bbox) for tid, bbox, _ in luggage_tracks}
        
        # Associate luggage with nearest person
        for lug_id, lug_center in luggage_centers.items():
            # Initialize luggage track if new
            if lug_id not in self.luggage_tracks:
                self.luggage_tracks[lug_id] = {
                    'owner_id': None,
                    'frames_alone': 0,
                    'first_seen': frame_count,
                    'last_owner_frame': frame_count,
                    'position': lug_center,
                    'abandoned': False
                }
            
            # Find nearest person
            min_distance = float('inf')
            nearest_person = None
            
            for person_id, person_center in person_centers.items():
                dist = euclidean_distance(lug_center, person_center)
                if dist < min_distance:
                    min_distance = dist
                    nearest_person = person_id
            
            # Update luggage track
            luggage_info = self.luggage_tracks[lug_id]
            luggage_info['position'] = lug_center
            
            if min_distance < self.distance_threshold:
                # Luggage has owner nearby
                luggage_info['owner_id'] = nearest_person
                luggage_info['frames_alone'] = 0
                luggage_info['last_owner_frame'] = frame_count
            else:
                # Luggage is alone
                luggage_info['frames_alone'] += 1
                
                # Check if abandoned (alone for threshold duration)
                frames_threshold = self.time_threshold * self.fps / CONFIG['PROCESS_EVERY_N_FRAMES']
                
                if luggage_info['frames_alone'] > frames_threshold and not luggage_info['abandoned']:
                    luggage_info['abandoned'] = True
                    
                    # Create alert
                    alert = {
                        'luggage_id': lug_id,
                        'position': lug_center,
                        'duration': luggage_info['frames_alone'] / (self.fps / CONFIG['PROCESS_EVERY_N_FRAMES']),
                        'frame': frame_count,
                        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    self.abandoned_alerts.append(alert)
                    print(f"🚨 ALERT: Abandoned luggage detected! ID: {lug_id}, Position: {lug_center}")
        
        # Clean up old tracks
        active_luggage_ids = set(luggage_centers.keys())
        ids_to_remove = [lid for lid in self.luggage_tracks.keys() 
                        if lid not in active_luggage_ids]
        for lid in ids_to_remove:
            del self.luggage_tracks[lid]
    
    def get_abandoned_luggage(self):
        """Get list of currently abandoned luggage"""
        return [(lid, info['position']) 
                for lid, info in self.luggage_tracks.items() 
                if info['abandoned']]
    
    def get_all_alerts(self):
        """Get all abandoned luggage alerts"""
        return self.abandoned_alerts

print("✅ Abandoned luggage detector defined!")


✅ Abandoned luggage detector defined!


In [28]:
# ================================
# INITIALIZE SYSTEM
# ================================

print("🚀 Initializing Airport Security System...")

# Load YOLO model
print(f"📦 Loading {CONFIG['MODEL']}...")
model = YOLO(CONFIG['MODEL'])
print("✅ Model loaded!")

# Open video
cap = cv2.VideoCapture(CONFIG['VIDEO_PATH'])
if not cap.isOpened():
    raise ValueError(f"❌ Cannot open video: {CONFIG['VIDEO_PATH']}")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"📹 Video: {frame_width}x{frame_height} @ {fps} FPS")
print(f"📊 Total frames: {total_frames}")

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(CONFIG['OUTPUT_PATH'], fourcc, fps, 
                      (frame_width, frame_height))

# Initialize components
tracker = SimpleTracker(
    max_age=CONFIG['MAX_AGE'],
    min_hits=CONFIG['MIN_HITS'],
    iou_threshold=CONFIG['IOU_THRESHOLD']
)

crowd_analyzer = CrowdDensityAnalyzer(
    frame_shape=(frame_height, frame_width, 3),
    grid_size=CONFIG['GRID_SIZE'],
    pixels_per_meter=CONFIG['PIXELS_PER_METER']
)

abandoned_detector = AbandonedLuggageDetector(
    distance_threshold=CONFIG['OWNER_DISTANCE_THRESHOLD'],
    time_threshold=CONFIG['STATIONARY_TIME'],
    fps=fps
)

# Storage for trajectories
trajectories = defaultdict(lambda: deque(maxlen=30))

# Statistics
stats = {
    'total_persons': 0,
    'total_luggage': 0,
    'overcrowding_alerts': 0,
    'abandoned_alerts': 0,
    'frames_processed': 0
}

print("✅ System initialized!")
print("="*60)


🚀 Initializing Airport Security System...
📦 Loading yolov8n.pt...
✅ Model loaded!
📹 Video: 3840x2160 @ 30 FPS
📊 Total frames: 222
📊 Grid initialized: 5x5
📏 Cell size: 768x432 pixels
✅ System initialized!


In [29]:
# ================================
# MAIN PROCESSING LOOP
# ================================

print("🎬 Starting video processing...")
print("Press 'q' to stop early")
print("="*60)

frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    
    # Process every N frames
    if frame_count % CONFIG['PROCESS_EVERY_N_FRAMES'] != 0:
        continue
    
    # Create copies for different visualizations
    display_frame = frame.copy()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # ============================================
    # STEP 1: DETECTION
    # ============================================
    results = model(frame, conf=CONFIG['CONFIDENCE_THRESHOLD'], verbose=False)
    
    detections_persons = []
    detections_luggage = []
    
    if len(results) > 0 and results[0].boxes is not None:
        boxes = results[0].boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())
            
            detection = [x1, y1, x2, y2, conf, cls]
            
            if cls == CONFIG['PERSON_CLASS']:
                detections_persons.append(detection)
            elif cls in CONFIG['LUGGAGE_CLASSES']:
                detections_luggage.append(detection)
    
    # ============================================
    # STEP 2: TRACKING
    # ============================================
    # Track persons
    person_tracks = tracker.update(detections_persons)
    
    # Track luggage (separate tracker instance would be better, but simplifying)
    luggage_tracks = []
    for det in detections_luggage:
        luggage_tracks.append((len(luggage_tracks), det[:4], int(det[5])))
    
    stats['total_persons'] = len(person_tracks)
    stats['total_luggage'] = len(luggage_tracks)
    
    # ============================================
    # STEP 3: CROWD DENSITY ANALYSIS
    # ============================================
    crowd_analyzer.reset_grid()
    
    person_centers = []
    for track_id, bbox, cls in person_tracks:
        center = get_center(bbox)
        person_centers.append(center)
        trajectories[track_id].append(center)
        crowd_analyzer.update_density([center])
    
    # Check overcrowding
    overcrowding_alerts = crowd_analyzer.check_overcrowding()
    if overcrowding_alerts:
        stats['overcrowding_alerts'] += len(overcrowding_alerts)
    
    # ============================================
    # STEP 4: OPTICAL FLOW
    # ============================================
    flow = None
    if CONFIG['ENABLE_OPTICAL_FLOW']:
        flow = crowd_analyzer.calculate_optical_flow(gray_frame)
    
    # ============================================
    # STEP 5: ABANDONED LUGGAGE DETECTION
    # ============================================
    abandoned_detector.update(person_tracks, luggage_tracks, frame_count)
    abandoned_luggage = abandoned_detector.get_abandoned_luggage()
    stats['abandoned_alerts'] = len(abandoned_detector.get_all_alerts())
    
    # ============================================
    # VISUALIZATION
    # ============================================
    
    # Draw person bounding boxes and IDs
    for track_id, bbox, cls in person_tracks:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        draw_text_with_background(display_frame, f"ID:{track_id}", 
                                  (x1, y1-10), bg_color=(0, 255, 0))
    
    # Draw luggage bounding boxes
    for lug_id, bbox, cls in luggage_tracks:
        x1, y1, x2, y2 = map(int, bbox)
        color = (0, 165, 255)  # Orange for luggage
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)
        draw_text_with_background(display_frame, "Luggage", 
                                  (x1, y1-10), bg_color=color)
    
    # Draw abandoned luggage alerts
    for lug_id, position in abandoned_luggage:
        cx, cy = position
        cv2.circle(display_frame, (cx, cy), 50, (0, 0, 255), 3)
        draw_text_with_background(display_frame, "ABANDONED!", 
                                  (cx-40, cy-60), 
                                  font_scale=0.8, 
                                  bg_color=(0, 0, 255))
        # Flashing effect
        if frame_count % 20 < 10:
            cv2.circle(display_frame, (cx, cy), 55, (0, 0, 255), 5)
    
    # Draw trajectories
    if CONFIG['SHOW_TRAJECTORIES']:
        for track_id, trail in trajectories.items():
            if len(trail) > 1:
                points = np.array(trail, dtype=np.int32)
                cv2.polylines(display_frame, [points], False, (255, 0, 255), 2)
    
    # Draw overcrowding alerts
    for alert in overcrowding_alerts:
        zone_i, zone_j = alert['zone']
        x = zone_j * crowd_analyzer.cell_width
        y = zone_i * crowd_analyzer.cell_height
        cv2.rectangle(display_frame, 
                     (x, y), 
                     (x + crowd_analyzer.cell_width, y + crowd_analyzer.cell_height),
                     (0, 0, 255), 3)
        draw_text_with_background(display_frame, 
                                 f"OVERCROWDING! {alert['count']} people", 
                                 (x+10, y+30),
                                 bg_color=(0, 0, 255))
    
    # Add statistics overlay
    stats_text = [
        f"Frame: {frame_count}/{total_frames}",
        f"Persons: {stats['total_persons']}",
        f"Luggage: {stats['total_luggage']}",
        f"Overcrowding Alerts: {stats['overcrowding_alerts']}",
        f"Abandoned Alerts: {stats['abandoned_alerts']}"
    ]
    
    y_offset = 30
    for i, text in enumerate(stats_text):
        draw_text_with_background(display_frame, text, (10, y_offset + i*30),
                                 font_scale=0.7, bg_color=(0, 0, 0))
    
    # Write frame to output video
    out.write(display_frame)
    stats['frames_processed'] += 1
    
    # Display progress every 30 frames
    if frame_count % 30 == 0:
        progress = (frame_count / total_frames) * 100
        elapsed = time.time() - start_time
        fps_processing = frame_count / elapsed if elapsed > 0 else 0
        
        clear_output(wait=True)
        print(f"⏳ Progress: {progress:.1f}% | Frame: {frame_count}/{total_frames}")
        print(f"⚡ Processing Speed: {fps_processing:.1f} FPS")
        print(f"👥 Current Persons: {stats['total_persons']}")
        print(f"🧳 Current Luggage: {stats['total_luggage']}")
        print(f"🚨 Abandoned Alerts: {stats['abandoned_alerts']}")
        print(f"⚠️  Overcrowding Alerts: {stats['overcrowding_alerts']}")

# Clean up
cap.release()
out.release()

elapsed_time = time.time() - start_time
print("\n" + "="*60)
print("✅ Processing complete!")
print(f"⏱️  Total time: {elapsed_time:.2f} seconds")
print(f"⚡ Average FPS: {stats['frames_processed']/elapsed_time:.2f}")
print(f"💾 Output saved to: {CONFIG['OUTPUT_PATH']}")
print(f"📊 Final Stats:")
print(f"   - Frames processed: {stats['frames_processed']}")
print(f"   - Abandoned luggage alerts: {stats['abandoned_alerts']}")
print(f"   - Overcrowding alerts: {stats['overcrowding_alerts']}")
print("="*60)


⏳ Progress: 94.6% | Frame: 210/222
⚡ Processing Speed: 0.5 FPS
👥 Current Persons: 5
🧳 Current Luggage: 0
🚨 Abandoned Alerts: 0
⚠️  Overcrowding Alerts: 0

✅ Processing complete!
⏱️  Total time: 404.75 seconds
⚡ Average FPS: 0.55
💾 Output saved to: output_video.mp4
📊 Final Stats:
   - Frames processed: 222
   - Abandoned luggage alerts: 0
   - Overcrowding alerts: 0
